In [1]:
import pandas as pd
import json
import os
import time


In [2]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [3]:
#username = 'templivs'
username_list  = ['GilbertCollard','dav_dec','Carbongate','bcassoret',
                  'Electroversenet','thinkfree55', 'KlassLib','sauvonsleclimat']
data_path = 'multiusers/'
#get_tweets = python_tweets.get_user_timeline(screen_name = username,  
#                                           count = 200, include_rts = True)
if not os.path.isdir(data_path):
    os.mkdir(data_path)

In [4]:
def fill_retweet_info(tweet_dic,raw_retweet):
    tweet_dic['retweeted_from'].append(raw_retweet['user']['screen_name'])
    if raw_retweet['truncated']:
        full_text = raw_retweet['extended_tweet']['full_text']
    else:
        full_text = raw_retweet['full_text']
    return tweet_dic, full_text

In [5]:
def get_user_tweets(tweet_handle, username,count=200):
    # Collect tweets
    tweets_dic = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'retweet_count': [],
        'user_mentions': [], 'urls': [], 'hashtags': [], 'geo': [], 'retweeted_from': []}

    for raw_tweet in tweet_handle.get_user_timeline(screen_name = username,  
                                           count = count, include_rts = True, tweet_mode='extended'):
        # Meta data
        tweets_dic['user'].append(raw_tweet['user']['screen_name'])
        ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(raw_tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
        tweets_dic['date'].append(ts)
        tweets_dic['favorite_count'].append(raw_tweet['favorite_count'])
        tweets_dic['retweet_count'].append(raw_tweet['retweet_count'])    
        tweets_dic['user_mentions'].append([user['screen_name'] for user in raw_tweet['entities']['user_mentions']])
        tweets_dic['urls'].append([url['url'] for url in raw_tweet['entities']['urls']])
        tweets_dic['hashtags'].append([htg['text'] for htg in raw_tweet['entities']['hashtags']])
        #if raw_tweet['entities']['hashtags']:
        #    print([htg['text'] for htg in raw_tweet['entities']['hashtags']])
        tweets_dic['geo'].append(raw_tweet['geo'])
        
        # Handle text and retweet data
        if raw_tweet['truncated']:
            full_text = raw_tweet['extended_tweet']['full_text']
        else:
            full_text = raw_tweet['full_text']    
        if 'retweeted_status' in raw_tweet:
            tweets_dic, full_text = fill_retweet_info(tweets_dic,raw_tweet['retweeted_status'])
        else:
            tweets_dic['retweeted_from'].append(None)
        tweets_dic['text'].append(full_text)
    return tweets_dic
    

In [6]:
def get_mentions_edges(tweet_df):
    mention_df = pd.DataFrame(columns=['user','mention','weight'])
    row_list = []
    for idx,tweet in tweet_df.iterrows():
        user = tweet['user']
        mentions = tweet['user_mentions']
        hashtags = tweet['hashtags']
        for m in mentions:
            row_list.append({'user':user,'mention': m, 'weight': 1, 'hashtags': hashtags})
    mention_df = pd.DataFrame(row_list)
    if mention_df.empty:
        return None
    # this agg only works with pandas version >= 0.25
    mention_grouped = mention_df.groupby(['user','mention']).agg(weight=('weight',sum),
                                                                 hashtags=('hashtags', sum))#lambda x: list(x)))    
    mention_g_list = mention_df.groupby(['user','mention'])['hashtags'].apply(list)
    mention_grouped.reset_index(level=['user', 'mention'], inplace=True)
    return mention_grouped,mention_g_list

In [58]:
def collect_user_mention(username,python_tweets,data_path):
    tweets_dic = get_user_tweets(python_tweets,username,count=200)
    #print(tweets_dic['geo'])
    tweet_df = pd.DataFrame(tweets_dic)
    mention_grouped,mention_g_list = get_mentions_edges(tweet_df)
    return mention_grouped, mention_g_list

In [59]:
def create_user_edgelist(python_tweets, data_path, username, thres=3):
    # initial user
    print('Processing',username)
    try:
        mention_grouped,mgl = collect_user_mention(username,python_tweets,data_path)
    except:
        print('exception catched on user {} !!!!!!!!!!!!'.format(username))
        return
    mention_grouped.to_csv(data_path + username + '_mentions.csv')
    print('First user done')

    # Threshold for number of mentions
    print('Using threshold:',thres)

    for idx,row in mention_grouped.iterrows():
        #print('processing mention',idx)
        mention_name = row['mention']
        if row['weight'] < thres:
            continue
        try:
            mention_grouped,mgl = collect_user_mention(mention_name,python_tweets,data_path)
        except:
            print('exception catched on user {} !!!!!!!!!!!!'.format(username))
            continue
        if mention_grouped is not None:
            mentionfilename = data_path + mention_name + '_mentions' +'_t' +str(thres)+'.csv'
            print('Writing',mentionfilename)
            mention_grouped.to_csv(mentionfilename)

In [60]:
#data_path = 'GBR_data/'
#username_list = ['GBR_Data']
thres = 3
for user in username_list:
    create_user_edgelist(python_tweets, data_path, user, thres=thres)

Processing GilbertCollard
exception catched on user GilbertCollard !!!!!!!!!!!!
Processing dav_dec
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

...

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [10]:
import glob

edge_df = pd.DataFrame()
for filename in glob.glob(data_path + '*_mentions' +'_t' +str(thres)+ '.csv'):
    print(filename)
    new_edge_df = pd.read_csv(filename)
    edge_df = edge_df.append(new_edge_df)
    

multiusers/_lnnen_WT_mentions_t3.csv
multiusers/neorepublik_mentions_t3.csv
multiusers/Martin_Sellner_mentions_t3.csv
multiusers/Groupe_Atlantic_mentions_t3.csv
multiusers/ethalises_mentions_t3.csv
multiusers/Electroversenet_mentions_t3.csv
multiusers/PetitsMatinsFC_mentions_t3.csv
multiusers/Elisabeth_Borne_mentions_t3.csv
multiusers/atheistsplained_mentions_t3.csv
multiusers/enerpresse_mentions_t3.csv
multiusers/444Cin444_mentions_t3.csv
multiusers/pagechristine1_mentions_t3.csv
multiusers/maxim_k_mentions_t3.csv
multiusers/MajaSeifenblase_mentions_t3.csv
multiusers/SandrineTreiner_mentions_t3.csv
multiusers/JaouenClaude_mentions_t3.csv
multiusers/Akuo_Energy_mentions_t3.csv
multiusers/OBFSU_mentions_t3.csv
multiusers/MichaelBauer218_mentions_t3.csv
multiusers/fmbreon_mentions_t3.csv
multiusers/DawnTJ90_mentions_t3.csv
multiusers/markrudolph2701_mentions_t3.csv
multiusers/dejavuallday_mentions_t3.csv
multiusers/sandrafellous__mentions_t3.csv
multiusers/babgi_mentions_t3.csv
multiuser

In [ ]:
edge_df[edge_df['hashtags'].apply(lambda x : len(x.split()))>1]

In [11]:
import networkx as nx

DEGREE_MIN = 4

G = nx.from_pandas_edgelist(edge_df,source='user',target='mention', edge_attr=['weight','hashtags'])
print('Nb of nodes:',G.number_of_nodes())
# Drop
remove = [node for node,degree in dict(G.degree()).items() if degree < DEGREE_MIN]
G.remove_nodes_from(remove)
print('Nb of nodes after removing less connected nodes:',G.number_of_nodes())

Nb of nodes: 15970
Nb of nodes after removing less connected nodes: 1171


In [14]:
import community

In [20]:
#first compute the best partition
partition = community.best_partition(G)
nx.set_node_attributes(G,partition,name='community')
print('Nb of partitions:',max(partition.values()))

Nb of partitions: 8


In [21]:
graphname = 'multiusersgraph'
#graphname = 'GBRgraph'

In [22]:
# Save the graph
graphfilename = data_path + graphname + '_t' + str(thres) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
print('Wrote',graphfilename)

Wrote multiusers/multiusersgraph_t3_graph.gexf


In [43]:
import ast

In [44]:
# get the hashtags for each community and inter-communities
tags_dic = {}
for node1,node2,data in G.edges(data=True):
    n1_com = G.nodes[node1]['community']
    n2_com = G.nodes[node2]['community']
    new_key = str(n1_com) + str(n2_com)
    # Convert string to list
    x = ast.literal_eval(data['hashtags'])
    keywords = [n.strip() for n in x]
    #keywords = data['hashtags'][1:-1].split()
    if new_key not in tags_dic:
        tags_dic[new_key] = keywords
    else:
        tags_dic[new_key] += keywords   

In [54]:
from collections import Counter
for key in tags_dic:
    most_common = Counter(tags_dic[key]).most_common(5)
    print(key)
    print(most_common)

00
[('DonAlphonso', 107), ('bdk19', 10), ('Klimaschutz', 9), ('Pertsch', 8), ('Hmmm', 8)]
01
[('Klimanotstand', 1)]
02
[]
08
[('LinkeVerbieten', 1), ('Merkel', 1)]
03
[]
16
[('climat', 2), ('Chine', 2), ('USA', 2), ('G7Biarritz', 2), ('GIEC', 1)]
10
[('AFD', 2), ('BoltonFire', 1), ('NPD', 1), ('Systempartei', 1), ('BRDGmbH', 1)]
11
[('opcaf', 177), ('chemtrails', 75), ('geoengineering', 64), ('GeoEngineering', 46), ('STUDY', 25)]
06
[('ConseildesMinistres', 1), ('FridaysForFuture', 1), ('ClimateStrike', 1), ('Libra', 1), ('climatechange', 1)]
22
[('fixedit', 41), ('ClimateFraud', 32), ('Fail', 18), ('MeToo', 3), ('LinearMind', 2)]
20
[]
18
[]
12
[]
13
[('BreakingNews', 1), ('TRUMP', 1), ('paris', 1), ('Acte53', 1), ('GiletsJaunes', 1)]
15
[('WEATHERWARFARE', 2), ('ANALYSIS', 2), ('US', 1), ('AccordDeParis', 1), ('émissions', 1)]
14
[]
04
[('Taiwan', 1), ('WWS', 1), ('WindWaterSolar', 1)]
33
[('Macron', 111), ('GiletsJaunes', 83), ('Paris', 57), ('LaMatinaleLCI', 54), ('Europe1', 50)]
4

In [57]:
for key in ['00','11','22','33','44','55','66','77','88']:
    most_common = Counter(tags_dic[key]).most_common(5)
    print('Community',key[0])
    print(most_common)

Community 0
[('DonAlphonso', 107), ('bdk19', 10), ('Klimaschutz', 9), ('Pertsch', 8), ('Hmmm', 8)]
Community 1
[('opcaf', 177), ('chemtrails', 75), ('geoengineering', 64), ('GeoEngineering', 46), ('STUDY', 25)]
Community 2
[('fixedit', 41), ('ClimateFraud', 32), ('Fail', 18), ('MeToo', 3), ('LinearMind', 2)]
Community 3
[('Macron', 111), ('GiletsJaunes', 83), ('Paris', 57), ('LaMatinaleLCI', 54), ('Europe1', 50)]
Community 4
[('nucléaire', 60), ('FRANCE', 55), ('nucléaires', 40), ('URANIUM', 37), ('NORVÈGE', 27)]
Community 5
[('climat', 60), ('ConventionCitoyenne', 60), ('nucléaire', 56), ('Paris', 26), ('CO2', 25)]
Community 6
[('EnR', 187), ('fnccr2019', 185), ('énergie', 159), ('PLF2020', 136), ('éolien', 101)]
Community 7
[('Nantes', 6), ('Utopiales2019', 4), ('Morrison', 4), ('MeToo', 3), ('VendrediFiction', 3)]
Community 8
[('snow', 4), ('SNOWFALL', 3), ('Livigno', 3), ('FRANCE', 2), ('hailstorm', 2)]


In [65]:
community_nodes = {}
for node,data in G.nodes(data=True):
    community = data['community']
    if  community not in community_nodes:
        community_nodes[community] = [(node, G.degree(node))]
    else:
        community_nodes[community].append((node, G.degree(node)))

In [72]:
ddf = pd.DataFrame(community_nodes[0],columns=['User','Degree'])

In [75]:
ddf.sort_values(by='Degree',ascending=False).head(20)

,User,Degree
36,stefanolix,65
9,Hallaschka_HH,62
67,markrudolph2701,59
19,_donalphonso,50
96,OlliLuksic,47
129,HauptmannF,47
0,_lnnen_WT,45
20,atheistsplained,45
90,MichaelBauer218,43
5,Atomreisfleisch,32


In [78]:
with pd.ExcelWriter(data_path + 'users.xlsx') as writer:
    for community in community_nodes:
        ddf = pd.DataFrame(community_nodes[community],columns=['User','Degree'])
        ddf = ddf.sort_values(by='Degree',ascending=False)#.head(20)
        ddf.to_excel(writer, sheet_name='Community_' + str(community),index=False)